# Computational Systems Biology
# HW2

In this homework, you study different kinds of reaction couplings in a metabolic network. Regardless of the biological intuition behind the couplings, these relations help us speed up exponential-time MILP problems in metabolic networks, including the most important problems in this field. After that, you will use QFCA to find a quantitative equation for a flux according to a proper set of irreversible fluxes.

In [3]:
import cobra

In this HW, you are allowed to use built-in cobra functions for finding blocked and essential reactions, which are `cobra.flux_analysis.find_blocked_reactions(model)` and `cobra.flux_analysis.find_essential_reactions(model)`, respectively. The first one returns a list of ids, but the second one returns a list of Reaction objects!

To define new variables to the model other then reactions, you can use `new_var = model.problem.Variable(name, lb=None, ub=None, type='continuous')`. The variable type can be `'continuous'`, `'integer'`, or `'binary'`. To add your variable to the model, use `model.add_cons_vars([new_var])`. Note that to use these variables in taylored constraints, you should use them without any `.flux_expression`. In fact, for a `cobra.Reaction` object, `reaction.flux_expression` returns the Variable related to that object, but here we are using a `Variable` object directly.

# Task 1: Coupling

For each pairs of reactions, $j_1$ and $j_2$, five possible coupling cases is imaginable:
1. Fully Coupled ($v_1 \Leftrightarrow v_2$): $R_{min} = R_{max} = c > 0$
2. Directionally Coupled ($v_1 \rightarrow v_2$): $R_{min} = 0, R_{max} = c > 0$
3. Directionally Coupled ($v_1 \leftarrow v_2$): $R_{min} = c > 0, R_{max} = \infty$
4. Partially Coupled ($v_1 \leftrightarrow v_2$): $R_{min} = c_1 > 0, R_{max} = c_2 > 0$
5. Uncoupled: $R_{min} = 0, R_{max} = \infty$

for $R = \frac{v_{j_1}}{v_{j_2}}$ and $J$ is the set of reactions. Finding $R_{min}$ and $R_{max}$ is based on solving the following optimization problem (a.k.a. FCF):

$$
minimize/maximize  \quad R = \frac{v_{j_1}}{v_{j_2}} \\
s.t.: \qquad \quad Sv = 0 \\
\qquad \qquad \qquad \qquad l_j \leq v_j \leq u_j \qquad j \in J \\
\qquad \qquad \qquad \qquad v_j \geq 0 \qquad j \in J.
$$ 

Note that in this formulation, all reversible reactions ($A \leftrightarrow B$) are replaced by two irreversible reactions 
($A \rightarrow B$ and $A \leftarrow B$) to prevent analytical problems for the fraction $\frac{v_{j_1}}{v_{j_2}}$. It would be unnecessary if you handle this concern differently.

## Part 1:

By transforming this problem to an LP, find all the irreversible reactions directionally coupled to biomass reaction (i.e., biomass $\rightarrow v_j$) in our `iAF1260` E.Coli metabolic network in the aerobic medium (consider fully and partially couplings as a special case of directional coupling). Compare the result to irreversible essential reactions and save the reactions which belong to one of these sets and not the other (i.e., the essential reactions which are not directionally coupled to biomass and vice versa). Be careful to consider off_reactions in aerobic conditions like in HW1. 

**Note:** Your equivalent LP can be in a different format than an FBA problem (but it shouldn't be drastically different!). It is not compulsory to use the COBRA toolbox for solving your LP problem, but here are some hints to keep using it. You learned how to add new variables to the model (dummy reaction is also a choice). To add any constraints to your model, you can use `new_constraint = model.problem.Constraint(rxn.flux_expression, lb=min_val, ub=max_val)` (one of the `lb` or `ub` arguments could be ignored or set to `None`), then `model.add_cons_vars([new_constraint])`. Review the tutorial in HW1 for more details. If any of your reaction variables do not contain lower and upper bounds, simply set those bounds to great negative and positive values. Consider `M = 1000000` or any greater number as infinity wherever necessary.

In [107]:
task1_model = cobra.io.read_sbml_model("iAF1260.xml")

In [108]:
# including off blocked reactions
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task1_model.remove_reactions([task1_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1]))])
    
for reaction in cobra.flux_analysis.find_blocked_reactions(task1_model):
    task1_model.remove_reactions([reaction])

In [4]:
# print all blocked reactions
Bl_reactions = []

for reaction in task1_model.reactions:
    if reaction.lower_bound == 0 and reaction.upper_bound == 0:
        Bl_reactions.append(reaction.id)
    
Bl_reactions

[]

In [48]:
# find all irreversible reactions
Ir_reactions = []

for reaction in task1_model.reactions:
    if reaction.lower_bound == 0 and reaction.upper_bound != 0:
        Ir_reactions.append(reaction)
    
Ir_reactions

[<Reaction 12DGR120tipp at 0x7facf324eaf0>,
 <Reaction 12DGR140tipp at 0x7facf30d5e50>,
 <Reaction 12DGR141tipp at 0x7facf333bca0>,
 <Reaction 12DGR160tipp at 0x7facf333be80>,
 <Reaction 12DGR161tipp at 0x7facf3226cd0>,
 <Reaction 12DGR180tipp at 0x7facf333bc70>,
 <Reaction 12DGR181tipp at 0x7facf333bd60>,
 <Reaction 14GLUCANabcpp at 0x7facf333be50>,
 <Reaction 14GLUCANtexi at 0x7facf333bcd0>,
 <Reaction 23DAPPAt2pp at 0x7facf3339a60>,
 <Reaction 23PDE2pp at 0x7facf3339be0>,
 <Reaction 23PDE4pp at 0x7facf3339c70>,
 <Reaction 23PDE7pp at 0x7facf3339d60>,
 <Reaction 23PDE9pp at 0x7facf3339e50>,
 <Reaction 2AGPA120tipp at 0x7facf3339fa0>,
 <Reaction 2AGPA140tipp at 0x7facf3339640>,
 <Reaction 2AGPA141tipp at 0x7facf3339c40>,
 <Reaction 2AGPA160tipp at 0x7facf3339cd0>,
 <Reaction 2AGPA161tipp at 0x7facf3339dc0>,
 <Reaction 2AGPA180tipp at 0x7facf3339130>,
 <Reaction 2AGPA181tipp at 0x7facf3339e20>,
 <Reaction 2AGPE120tipp at 0x7facf3339bb0>,
 <Reaction 2AGPE140tipp at 0x7facf3339f10>,
 <Re

In [125]:
# find all irreversible essential reactions
IrEs_reactions = []

for reaction in cobra.flux_analysis.find_essential_reactions(task2_consistent_model):
    IrEs_reactions.append(reaction.id)
    
IrEs_reactions

['EX_glc(e)', 'GLCtex_f']

In [50]:
biomass_reaction = task1_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

In [51]:
biomass_reaction.id

'Ec_biomass_iAF1260_WT_59p81M'

In [11]:
# convert a whole model to an irreversible model
print('Converting the model to irreversible one:')
for reaction in task1_model.reactions:
    if reaction.reversibility:
        reverse_reaction = cobra.core.reaction.copy(reaction)
        reverse_reaction.__imul__(-1)
        reverse_reaction.id = reaction.id + '_rv'
        reverse_reaction.lower_bound = 0
        reverse_reaction.upper_bound = -reaction.lower_bound
        reaction.lower_bound = 0
        task1_model.add_reaction(reverse_reaction)
        print('.', end='')
print('\nDone', end='')

Converting the model to irreversible one:
......................................................................................................................................................................................................................................................................................................................................
Done

In [9]:
# calcullate reaction bounds and make them all inf
Bounds = {}
for reaction in task1_model.reactions:
    if reaction.lower_bound == -1000:
        reaction.lower_bound = -1e6
    if reaction.upper_bound == 1000:
        reaction.upper_bound = 1e6
        
    Bounds[reaction.id] = (reaction.lower_bound, reaction.upper_bound)
    reaction.lower_bound = -1e6
    reaction.upper_bound = 1e6

In [10]:
# Add variable t abd bounds
t = task1_model.problem.Variable('t', lb=0, ub=1e6, type='continuous')
task1_model.add_cons_vars([t])

t_constraint = []
for reaction in task1_model.reactions:
    t_constraint.append(task1_model.problem.Constraint(reaction.flux_expression - Bounds[reaction.id][0]*t, lb=0, ub=1e6))
    t_constraint.append(task1_model.problem.Constraint(Bounds[reaction.id][1]*t - reaction.flux_expression, lb=0, ub=1e6))

task1_model.add_cons_vars(t_constraint)

In [11]:
len(Ir_reactions)

1138

In [12]:
# find all reactions directionally coupled to biomass reaction
Dc_reactions = []
counter = 0;

for reaction in Ir_reactions:

    reaction.upper_bound = 1
    reaction.lower_bound = 1
    
    task1_model.objective = biomass_reaction.id
    task1_model.objective_direction = 'max'
    max_flux = task1_model.optimize().objective_value
    t_max = task1_model.solver.variables.t.primal
    
    task1_model.objective = biomass_reaction.id
    task1_model.objective_direction = 'min'
    min_flux = task1_model.optimize().objective_value
    t_min = task1_model.solver.variables.t.primal

    print('Min flux: ', min_flux)
    print('Tmin: ', t_min)
    print('Max flux: ', max_flux)
    print('Tmax: ', t_max)

    if (min_flux ) >= -1 and (max_flux) < 700:
        Dc_reactions.append(reaction.id)
        
    reaction.upper_bound = 1e6
    reaction.lower_bound = -1e6
    
    counter += 1
    print(counter)

Min flux:  1.240166586225066e-16
Tmin:  0.9999586267950247
Max flux:  0.9098540988534659
Tmax:  0.9999363498935251
1
Min flux:  0.0
Tmin:  0.9999536604994074
Max flux:  0.9098540988430692
Tmax:  0.9999363498935249
2
Min flux:  0.0
Tmin:  0.9999531577119661
Max flux:  0.9098540988430777
Tmax:  0.999936349893525
3
Min flux:  -2.957413367286421e-16
Tmin:  0.9999536604994074
Max flux:  0.9098540988430783
Tmax:  0.999936349893525
4
Min flux:  0.0
Tmin:  0.999951570244996
Max flux:  0.9098540988430786
Tmax:  0.999936349893525
5
Min flux:  -4.3181271860859236e-16
Tmin:  0.9999513194137117
Max flux:  0.9098540988429767
Tmax:  0.999936349893525
6
Min flux:  8.595460835832793e-16
Tmin:  0.9999466557067779
Max flux:  0.9098540988430737
Tmax:  0.999936349893525
7
Min flux:  -5.637019700147416e-16
Tmin:  0.9999541860339609
Max flux:  0.9006962989401609
Tmax:  0.9999362885464423
8
Min flux:  0.0
Tmin:  0.9999396026190384
Max flux:  0.9006962989401672
Tmax:  0.9999362885464425
9
Min flux:  -2.1715772

/Users/omidsh/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Min flux:  0.0
Tmin:  0.9999766128711405
Max flux:  0.0
Tmax:  0.9999766128711405
46
Min flux:  3.303231415484851e-16
Tmin:  0.9999706179836979
Max flux:  0.6326638180389721
Tmax:  0.9999526273676288
47
Min flux:  0.0
Tmin:  0.9999766128711417
Max flux:  0.0
Tmax:  0.9999766128711417
48
Min flux:  0.0
Tmin:  0.9999709656666627
Max flux:  0.5832278789114814
Tmax:  0.9999553135471219
49
Min flux:  0.0
Tmin:  0.9999766128711405
Max flux:  0.0
Tmax:  0.9999766128711405
50
Min flux:  1.8379512803031876e-16
Tmin:  0.9999697955136844
Max flux:  0.5048785033202178
Tmax:  0.9999592303263455
51
Min flux:  0.9304589160457634
Tmin:  0.999936237940401
Max flux:  0.9304589160457634
Tmax:  0.999936237940401
52
Min flux:  -5.340494386245753e-16
Tmin:  0.9999729642311502
Max flux:  0.7983511877528275
Tmax:  0.9999435833918708
53
Min flux:  -4.560748993997698e-17
Tmin:  0.9999662307340863
Max flux:  0.7983511874739804
Tmax:  0.9999435833918918
54
Min flux:  -7.564532618818709e-17
Tmin:  0.99997286963090

Min flux:  5.413357980378886e-17
Tmin:  0.9999377018915631
Max flux:  0.8297238614450658
Tmax:  0.9999363630714836
121
Min flux:  1.827734128189535e-16
Tmin:  0.9999410959072413
Max flux:  0.8205660615421629
Tmax:  0.9999363017244012
122
Min flux:  -3.243368954398124e-16
Tmin:  0.9999408482210322
Max flux:  0.829723861445068
Tmax:  0.9999363630714835
123
Min flux:  8.805388771004178e-16
Tmin:  0.9999367837891511
Max flux:  0.8274346440840779
Tmax:  0.9999365977187724
124
Min flux:  1.2518636728979994e-15
Tmin:  0.999954069080546
Max flux:  0.8297238614450807
Tmax:  0.9999363630714836
125
Min flux:  0.0
Tmin:  0.9999547264476336
Max flux:  0.8297238614449328
Tmax:  0.9999363630714836
126
Min flux:  1.5364168466999025e-17
Tmin:  0.9999538172389197
Max flux:  0.8320130788060741
Tmax:  0.9999361284241948
127
Min flux:  0.0
Tmin:  0.9999788564837637
Max flux:  0.053639989814038225
Tmax:  0.9999777443421027
128
Min flux:  1.0237057373642402e-15
Tmin:  0.9999369390541311
Max flux:  0.87876793

Min flux:  1.8845927323250598e-16
Tmin:  0.9999456003970953
Max flux:  0.9213011161069903
Tmax:  0.9999361765933186
199
Min flux:  -1.3461226675657457e-16
Tmin:  0.9999494168791905
Max flux:  0.921301116106987
Tmax:  0.9999361765933186
200
Min flux:  1.0458091582706042e-15
Tmin:  0.9999607465811998
Max flux:  0.9213011161069757
Tmax:  0.9999361765933186
201
Min flux:  -3.979398413805888e-12
Tmin:  0.9999578624991008
Max flux:  0.9213011159429808
Tmax:  0.9999361765933198
202
Min flux:  -7.630185097050766e-17
Tmin:  0.999957862499101
Max flux:  0.9213011160992954
Tmax:  0.9999361765933188
203
Min flux:  0.9304589160109487
Tmin:  0.9999362379404007
Max flux:  0.9304589160109487
Tmax:  0.9999362379404007
204
Min flux:  2.543395032350255e-17
Tmin:  0.9999579308547775
Max flux:  0.891538499021909
Tmax:  0.9999362271993599
205
Min flux:  2.0448759949275924e-16
Tmin:  0.9999632380909192
Max flux:  0.8089763854324632
Tmax:  0.9999416981043098
206
Min flux:  -1.3902029862556575e-11
Tmin:  0.999

Min flux:  3.846687860723636e-17
Tmin:  0.9999544287805753
Max flux:  0.9121544930942294
Tmax:  0.9999361153211066
277
Min flux:  -1.490215569603265e-16
Tmin:  0.9999521031345557
Max flux:  0.9121433162281046
Tmax:  0.9999361152462362
278
Min flux:  0.0
Tmin:  0.9999728739300097
Max flux:  0.8481227071851093
Tmax:  0.9999402701540834
279
Min flux:  -1.7041091326659017e-11
Tmin:  0.9999784264406496
Max flux:  0.8481227071224148
Tmax:  0.9999402701540856
280
Min flux:  0.9304589160440856
Tmin:  0.9999362379404018
Max flux:  0.9304589160440856
Tmax:  0.9999362379404018
281
Min flux:  0.9304589160440856
Tmin:  0.9999362379404018
Max flux:  0.9304589160440856
Tmax:  0.9999362379404018
282
Min flux:  0.9304589160440856
Tmin:  0.9999362379404018
Max flux:  0.9304589160440856
Tmax:  0.9999362379404018
283
Min flux:  -4.882648556883439e-16
Tmin:  0.99997603854858
Max flux:  0.8930945431518957
Tmax:  0.9999369825292014
284
Min flux:  1.3299855995299144e-15
Tmin:  0.9999555604442241
Max flux:  0.

Min flux:  0.0
Tmin:  0.9999804946260691
Max flux:  0.9057223589011888
Tmax:  0.9999386068258318
353
Min flux:  6.22421774949909e-16
Tmin:  0.9999785029551015
Max flux:  0.8946573666546863
Tmax:  0.9999388358510684
354
Min flux:  0.0
Tmin:  0.9999789197645829
Max flux:  0.0
Tmax:  0.9999789197645829
355
Min flux:  -1.5585054033819995e-09
Tmin:  0.9999803737984683
Max flux:  0.8584450311782985
Tmax:  0.9999400580902513
356
Min flux:  -7.118063688724788e-09
Tmin:  0.999955365272848
Max flux:  0.8091784851062146
Tmax:  0.999935975457495
357
Min flux:  7.407957479365726e-18
Tmin:  0.9999695552680252
Max flux:  0.7606356757053783
Tmax:  0.9999369280316251
358
Min flux:  0.0
Tmin:  0.9999691808201254
Max flux:  0.8242424739969404
Tmax:  0.9999390570098398
359
Min flux:  -8.207982862633796e-18
Tmin:  0.9999781508297207
Max flux:  0.8674982044791396
Tmax:  0.9999398525242184
360
Min flux:  0.0
Tmin:  0.9999755327872121
Max flux:  0.7570528736953523
Tmax:  0.9999411469421672
361
Min flux:  2.48

OptimizationError: Solver status is 'undefined'.

In [268]:
len(Dc_reactions)

796

In [282]:
IrDc_reaction = Dc_reactions

In [276]:
Irreversible

['12DGR120tipp',
 '12DGR140tipp',
 '12DGR141tipp',
 '12DGR160tipp',
 '12DGR161tipp',
 '12DGR180tipp',
 '12DGR181tipp',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPEAT120',
 '2AGPEAT140',
 '2AGPEAT141',
 '2AGPEAT160',
 '2AGPEAT161',
 '2AGPEAT180',
 '2AGPEAT181',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 '2AGPGAT120',
 '2AGPGAT140',
 '2AGPGAT141',
 '2AGPGAT160',
 '2AGPGAT161',
 '2AGPGAT180',
 '2AGPGAT181',
 '2MAHMP',
 '3HAD100',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 '3HAD180',
 '3HAD181',
 '3HAD40',
 '3HAD60',
 '3HAD80',
 '3OAR121',
 '3OAR141',
 '3OAR161',
 '3OAR181',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS140',
 '3OAS141',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 '3OAS

In [277]:
IrEs_reactions_u = []
len(Irreversible)
for r in IrEs_reactions:
    if r.id in Irreversible:
        IrEs_reactions_u.append(r.id)
len(IrEs_reactions_u)

266

In [278]:
IrEs_reactions = IrEs_reactions_u

In [283]:
# find result
rxn_ids_list = []

for reaction in IrDc_reaction + IrEs_reactions:
    if  (reaction in IrDc_reaction and reaction not in IrEs_reactions) or (reaction in IrEs_reactions and reaction not in IrDc_reaction):
        rxn_ids_list.append(reaction)

In [279]:
len(IrEs_reactions)

266

In [284]:
len(rxn_ids_list)

628

In [43]:
# Save Your Results:
with open('Solutions/Task 1/task1_part1.txt', 'w') as f:
    for rxn_id in Ir_reactions:
        f.write("%s\n" % rxn_id.id)

In [285]:
# Save Your Results:
with open('Solutions/Task 1/task1_part1.txt', 'w') as f:
    for rxn_id in rxn_ids_list:
        f.write("%s\n" % rxn_id)

In [3]:
# Second Way of Solving the problem

In [46]:
task1_model = cobra.io.read_sbml_model("iAF1260.xml")

In [42]:
# including off blocked reactions
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task1_model.remove_reactions([task1_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1]))])
    
# for reaction in cobra.flux_analysis.find_blocked_reactions(task1_model):
#     task1_model.remove_reactions([reaction])

In [10]:
cobra.flux_analysis.find_blocked_reactions(task1_model)

[]

In [43]:
# print all blocked reactions
Bl_reactions = []

for reaction in task1_model.reactions:
    if reaction.lower_bound == 0 and reaction.upper_bound == 0:
        Bl_reactions.append(reaction.id)
    
Bl_reactions

['LDH_D_f', 'LDH_D2', 'L-LACD2', 'L-LACD3', 'Ec_biomass_iAF1260_core_59p81M']

In [11]:
# find all irreversible reactions
Ir_reactions = []

for reaction in task1_model.reactions:
    if reaction.lower_bound == 0 and reaction.upper_bound != 0:
        Ir_reactions.append(reaction)
    
Ir_reactions

[<Reaction 12DGR120tipp at 0x7facd3ec22b0>,
 <Reaction 12DGR140tipp at 0x7facd42fc0a0>,
 <Reaction 12DGR141tipp at 0x7facd42fcf10>,
 <Reaction 12DGR160tipp at 0x7facd42fe310>,
 <Reaction 12DGR161tipp at 0x7facd42fe460>,
 <Reaction 12DGR180tipp at 0x7facd42fe5b0>,
 <Reaction 12DGR181tipp at 0x7facd42fe700>,
 <Reaction 2AGPA120tipp at 0x7facd4300d00>,
 <Reaction 2AGPA140tipp at 0x7facd42feeb0>,
 <Reaction 2AGPA141tipp at 0x7facd4300730>,
 <Reaction 2AGPA160tipp at 0x7facd4300c70>,
 <Reaction 2AGPA161tipp at 0x7facd4300e80>,
 <Reaction 2AGPA180tipp at 0x7facd4300be0>,
 <Reaction 2AGPA181tipp at 0x7facd4301f10>,
 <Reaction 2AGPE120tipp at 0x7facd4301850>,
 <Reaction 2AGPE140tipp at 0x7facd4301c70>,
 <Reaction 2AGPE141tipp at 0x7facd4301b20>,
 <Reaction 2AGPE160tipp at 0x7facd4301dc0>,
 <Reaction 2AGPE161tipp at 0x7facd43019a0>,
 <Reaction 2AGPE180tipp at 0x7facd4301d90>,
 <Reaction 2AGPE181tipp at 0x7facd4301c40>,
 <Reaction 2AGPEAT120 at 0x7facd43039d0>,
 <Reaction 2AGPEAT140 at 0x7facd43

In [12]:
# find all irreversible essential reactions
IrEs_reactions = []

for reaction in cobra.flux_analysis.find_essential_reactions(task1_model):
    if reaction.lower_bound == 0 and reaction.upper_bound != 0:
        IrEs_reactions.append(reaction.id)
    
IrEs_reactions

['G5SADs',
 'HISTP',
 'PPND',
 'CYSTL',
 'ANS',
 'U23GAAT',
 'AMMQLT8',
 '3OAR161',
 'KDOPP',
 'DHPTDCs',
 'PPCDC',
 'DHNAOT4',
 'GCALDD',
 'UAAGDS',
 'AMPMS2',
 'ANPRT',
 'PPNCL2',
 'GLUTRS',
 'DASYN181',
 'ACODA',
 'DNMPPA',
 'THDPS',
 'DHNPA2',
 'DNTPPA',
 'DHPS2',
 'TDSK',
 'GLUTRR',
 'GLUPRT',
 'DHQTi',
 'TDPDRE',
 'GF6PTA',
 'KDOCT2',
 'DHQS',
 'HISTD',
 'NADS1',
 'CYSS',
 'TDPDRR',
 'KDOPS',
 'DASYN160',
 'TDPGDH',
 'DHPPDA2',
 'APRAUR',
 'DAPDC',
 'PPNDH',
 '3OAS141',
 'DPCOAK',
 'HMBS',
 'DASYN161',
 'DPR',
 'CBL1tonex',
 '3OAS100',
 'CTPS2',
 '3OAS80',
 'METAT',
 'CS',
 'OPHBDC',
 'PSSA181',
 'DB4PS',
 'P5CR',
 'ACLS',
 'ACGS',
 'OXGDC2',
 'PTPATi',
 'FMNAT',
 'ACGK',
 '3OAR181',
 'AIRC2',
 'AHCYSNS',
 '3OAS160',
 'PSSA161',
 'METS',
 'PSSA160',
 'USHD',
 'PGSA181',
 'S7PI',
 'GLCTR2',
 'GLCTR3',
 'OMCDC',
 'UNK3',
 'MTRK',
 'OMPDC',
 'SULabcpp',
 'GALT1',
 'NPHS',
 'UPP3MT',
 'OMBZLM',
 'UPP3S',
 'PMDPHT',
 'PPBNGS',
 'GLGC',
 'GRTT',
 'PMPK',
 'GLUCYS',
 'NADK',
 'SADT2',
 

In [13]:
biomass_reaction = task1_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

In [14]:
biomass_reaction.id

'Ec_biomass_iAF1260_WT_59p81M'

In [11]:
# convert a whole model to an irreversible model
print('Converting the model to irreversible one:')
for reaction in task1_model.reactions:
    if reaction.reversibility:
        reverse_reaction = cobra.core.reaction.copy(reaction)
        reverse_reaction.__imul__(-1)
        reverse_reaction.id = reaction.id + '_rv'
        reverse_reaction.lower_bound = 0
        reverse_reaction.upper_bound = -reaction.lower_bound
        reaction.lower_bound = 0
        task1_model.add_reaction(reverse_reaction)
        print('.', end='')
print('\nDone', end='')

Converting the model to irreversible one:
......................................................................................................................................................................................................................................................................................................................................
Done

In [9]:
# calcullate reaction bounds and make them all inf
Bounds = {}
for reaction in task1_model.reactions:
    if reaction.lower_bound == -1000:
        reaction.lower_bound = -1e6
    if reaction.upper_bound == 1000:
        reaction.upper_bound = 1e6
        
    Bounds[reaction.id] = (reaction.lower_bound, reaction.upper_bound)
    reaction.lower_bound = -1e6
    reaction.upper_bound = 1e6

In [10]:
# Add variable t abd bounds
t = task1_model.problem.Variable('t', lb=0, ub=1e6, type='continuous')
task1_model.add_cons_vars([t])

t_constraint = []
for reaction in task1_model.reactions:
    t_constraint.append(task1_model.problem.Constraint(reaction.flux_expression - Bounds[reaction.id][0]*t, lb=0, ub=1e6))
    t_constraint.append(task1_model.problem.Constraint(Bounds[reaction.id][1]*t - reaction.flux_expression, lb=0, ub=1e6))

task1_model.add_cons_vars(t_constraint)

In [16]:
len(Ir_reactions)

1138

In [21]:
# find all reactions directionally coupled to biomass reaction
Dc_reactions = []
counter = 0;

for reaction in Ir_reactions:
    
    ub = reaction.upper_bound
    lb = reaction.lower_bound
    
    reaction.lower_bound = 0    
    reaction.upper_bound = 0
    
    task1_model.objective = biomass_reaction.id
    task1_model.objective_direction = 'max'
    max_flux = task1_model.optimize().objective_value
    
    task1_model.objective = biomass_reaction.id
    task1_model.objective_direction = 'min'
    min_flux = task1_model.optimize().objective_value

    print('Min flux: ', min_flux)
    print('Max flux: ', max_flux)

    if (min_flux) > -1e-6 and (max_flux) < 1e-6:
        Dc_reactions.append(reaction.id)
        
    reaction.lower_bound = lb    
    reaction.upper_bound = ub
    
    counter += 1
    print(counter)

Min flux:  0.0
Max flux:  0.9302079746054036
1
Min flux:  0.0
Max flux:  0.9302079746054039
2
Min flux:  0.0
Max flux:  0.9302079746054039
3
Min flux:  0.0
Max flux:  0.9302079746054062
4
Min flux:  0.0
Max flux:  0.9302079746054055
5
Min flux:  0.0
Max flux:  0.9302079746054057
6
Min flux:  0.0
Max flux:  0.930207974605393
7
Min flux:  0.0
Max flux:  0.9302079746053936
8
Min flux:  0.0
Max flux:  0.9302079746053931
9
Min flux:  0.0
Max flux:  0.930207974605399
10
Min flux:  0.0
Max flux:  0.9302079746053978
11
Min flux:  0.0
Max flux:  0.9302079746054044
12
Min flux:  0.0
Max flux:  0.930207974605404
13
Min flux:  0.0
Max flux:  0.9302079746054034
14
Min flux:  0.0
Max flux:  0.9302079746054046
15
Min flux:  0.0
Max flux:  0.9302079746054049
16
Min flux:  0.0
Max flux:  0.9302079746054048
17
Min flux:  0.0
Max flux:  0.9302079746054103
18
Min flux:  0.0
Max flux:  0.9302079746054107
19
Min flux:  0.0
Max flux:  0.9302079746054103
20
Min flux:  0.0
Max flux:  0.9302079746054113
21
Min 

Min flux:  0.0
Max flux:  0.9302079746054022
175
Min flux:  0.0
Max flux:  0.9302079746054024
176
Min flux:  0.0
Max flux:  0.9302079746054074
177
Min flux:  0.0
Max flux:  0.9282178493401197
178
Min flux:  1.8975774685389473e-31
Max flux:  1.8975774685389473e-31
179
Min flux:  0.0
Max flux:  -1.8975774685389473e-31
180
Min flux:  0.0
Max flux:  0.9302079746054103
181
Min flux:  0.0
Max flux:  0.930207974607431
182
Min flux:  0.0
Max flux:  0.9302079746054077
183
Min flux:  0.0
Max flux:  0.9302079746054183
184
Min flux:  0.0
Max flux:  0.9302079746054192
185
Min flux:  0.0
Max flux:  0.9302079746054217
186
Min flux:  0.0
Max flux:  4.6338460398268695e-30
187
Min flux:  0.0
Max flux:  -3.0188310537379013e-12
188
Min flux:  0.0
Max flux:  0.9302079746054143
189
Min flux:  0.0
Max flux:  0.9302079746054054
190
Min flux:  -1.2408926646553315e-30
Max flux:  -1.2408926646553315e-30
191
Min flux:  -1.2408926646553315e-30
Max flux:  -1.2408926646553315e-30
192
Min flux:  0.0
Max flux:  0.9302

Min flux:  0.0
Max flux:  0.9302079746054072
340
Min flux:  0.0
Max flux:  8.512935718202953e-16
341
Min flux:  0.0
Max flux:  8.512935718202944e-16
342
Min flux:  0.0
Max flux:  0.9302079746054086
343
Min flux:  0.0
Max flux:  0.9302079746054089
344
Min flux:  0.0
Max flux:  0.930207974605408
345
Min flux:  0.0
Max flux:  0.0
346
Min flux:  0.0
Max flux:  0.9302079746054197
347
Min flux:  0.0
Max flux:  0.9302079746054184
348
Min flux:  0.0
Max flux:  0.930207974605418
349
Min flux:  0.0
Max flux:  0.9302079746054182
350
Min flux:  0.0
Max flux:  0.9302079746054186
351
Min flux:  0.0
Max flux:  0.9302079746054184
352
Min flux:  0.0
Max flux:  0.9300363704957864
353
Min flux:  0.0
Max flux:  0.9302079746054018
354
Min flux:  0.0
Max flux:  0.9302079746054019
355
Min flux:  0.0
Max flux:  0.9302079746054066
356
Min flux:  0.0
Max flux:  0.9302079746054048
357
Min flux:  0.0
Max flux:  0.9302079746054033
358
Min flux:  0.0
Max flux:  0.9302079746054045
359
Min flux:  0.0
Max flux:  0.930

Min flux:  0.0
Max flux:  1.5180619748311593e-30
518
Min flux:  0.0
Max flux:  0.9302079746053977
519
Min flux:  0.0
Max flux:  0.0
520
Min flux:  0.0
Max flux:  0.9283823781978319
521
Min flux:  0.0
Max flux:  0.9302079746054052
522
Min flux:  0.0
Max flux:  0.9302079746054047
523
Min flux:  0.0
Max flux:  -3.6474088893357034e-15
524
Min flux:  0.0
Max flux:  6.213789082259324e-32
525
Min flux:  0.0
Max flux:  0.9302079746054054
526
Min flux:  0.0
Max flux:  0.0
527
Min flux:  0.0
Max flux:  0.9302079746054046
528
Min flux:  0.0
Max flux:  0.9302079746054053
529
Min flux:  0.0
Max flux:  0.9302079746054077
530
Min flux:  0.0
Max flux:  4.751103018758768e-16
531
Min flux:  0.0
Max flux:  0.9302079746054094
532
Min flux:  0.0
Max flux:  0.9302079746054019
533
Min flux:  0.0
Max flux:  0.930207974605407
534
Min flux:  0.0
Max flux:  0.9302079746054134
535
Min flux:  0.0
Max flux:  0.9302079746054112
536
Min flux:  0.0
Max flux:  0.9302079746054078
537
Min flux:  0.0
Max flux:  0.93020797

Min flux:  0.0
Max flux:  0.9302079746054056
694
Min flux:  0.0
Max flux:  0.9302079746054054
695
Min flux:  0.0
Max flux:  0.9302079746054059
696
Min flux:  0.0
Max flux:  0.9302079746054058
697
Min flux:  0.0
Max flux:  0.9302079746054055
698
Min flux:  0.0
Max flux:  0.9302079746054059
699
Min flux:  0.0
Max flux:  0.9302079746054055
700
Min flux:  0.0
Max flux:  0.9302079746054083
701
Min flux:  0.0
Max flux:  0.9302079746054102
702
Min flux:  0.0
Max flux:  0.9302079746054082
703
Min flux:  0.0
Max flux:  0.930207974605409
704
Min flux:  0.0
Max flux:  0.9302079746054095
705
Min flux:  0.0
Max flux:  0.9302079746054107
706
Min flux:  0.0
Max flux:  0.9302079746054107
707
Min flux:  0.0
Max flux:  0.9302079746054108
708
Min flux:  0.0
Max flux:  0.9302079746054102
709
Min flux:  0.0
Max flux:  0.9302079746054038
710
Min flux:  0.0
Max flux:  3.640979754042891e-30
711
Min flux:  0.0
Max flux:  0.9301678883694694
712
Min flux:  0.0
Max flux:  7.1618788786363656e-31
713
Min flux:  0.0

Min flux:  0.0
Max flux:  0.9302079746054055
865
Min flux:  0.0
Max flux:  0.0
866
Min flux:  0.0
Max flux:  0.9302079746054103
867
Min flux:  0.0
Max flux:  0.9302079746054166
868
Min flux:  0.0
Max flux:  0.930207974605405
869
Min flux:  0.0
Max flux:  0.795990233659664
870
Min flux:  0.0
Max flux:  0.9302046513381823
871
Min flux:  0.0
Max flux:  0.9302079746054052
872
Min flux:  0.0
Max flux:  0.0
873
Min flux:  0.0
Max flux:  0.930207974605406
874
Min flux:  0.0
Max flux:  0.9302079746053963
875
Min flux:  0.0
Max flux:  0.9302079746053978
876
Min flux:  0.0
Max flux:  0.9302079746053977
877
Min flux:  0.0
Max flux:  0.0
878
Min flux:  0.0
Max flux:  0.0
879
Min flux:  0.0
Max flux:  0.9302079746054142
880
Min flux:  0.0
Max flux:  1.2021989606474408e-16
881
Min flux:  0.0
Max flux:  0.9302079746053206
882
Min flux:  0.0
Max flux:  0.9302079746054257
883
Min flux:  0.0
Max flux:  0.9152689777616156
884
Min flux:  0.0
Max flux:  0.9302079746054119
885
Min flux:  0.0
Max flux:  0.93

Min flux:  0.0
Max flux:  0.9302079746054064
1042
Min flux:  0.0
Max flux:  0.930207974605411
1043
Min flux:  0.0
Max flux:  0.9302079746054055
1044
Min flux:  2.7302894936306675e-31
Max flux:  2.7302894936306675e-31
1045
Min flux:  2.7302894936306675e-31
Max flux:  2.7302894936306675e-31
1046
Min flux:  2.7302894936306675e-31
Max flux:  2.7302894936306675e-31
1047
Min flux:  0.0
Max flux:  8.190868480892003e-31
1048
Min flux:  -1.1196016084793164e-29
Max flux:  -1.1196016084793164e-29
1049
Min flux:  0.0
Max flux:  0.9302079746054158
1050
Min flux:  0.0
Max flux:  0.0
1051
Min flux:  0.0
Max flux:  0.9302079746054127
1052
Min flux:  0.0
Max flux:  0.9302079746054154
1053
Min flux:  0.0
Max flux:  0.9302079746054065
1054
Min flux:  0.0
Max flux:  0.0
1055
Min flux:  0.0
Max flux:  0.0
1056
Min flux:  0.0
Max flux:  0.9302079746054078
1057
Min flux:  0.0
Max flux:  0.9302079746054109
1058
Min flux:  0.0
Max flux:  0.8741986389366468
1059
Min flux:  0.0
Max flux:  -1.3098722361338148e-28

In [19]:
print(Dc_reactions)

['3HAD100', '3HAD120', '3HAD121', '3HAD140', '3HAD141', '3HAD160', '3HAD161', '3HAD181', '3HAD40', '3HAD60', '3HAD80', '3OAR121', '3OAR141', '3OAR161', '3OAR181', '3OAS100', '3OAS120', '3OAS121', '3OAS140', '3OAS141', '3OAS160', '3OAS161', '3OAS181', '3OAS60', '3OAS80', 'ACCOAC', 'ACGK', 'ACGS', 'ACHBS', 'ACLS', 'ACODA', 'ADCL', 'ADCS', 'ADMDC', 'ADSK', 'ADSS', 'AGMHE', 'AGPAT160', 'AGPAT161', 'AGPAT181', 'AHCYSNS', 'AIRC2', 'AMMQLT8', 'AMPMS2', 'ANPRT', 'ANS', 'APRAUR', 'ARGSS', 'ASP1DC', 'ASPCT', 'ATPPRT', 'BPNT', 'CBL1abcpp', 'CBL1tonex', 'CDPMEK', 'CHORM', 'CHORS', 'CHRPL', 'CLt3_2pp', 'COBALT2tpp', 'COLIPAabcpp', 'COLIPAtex', 'CS', 'CTPS2', 'CU2tpp', 'CYSS', 'CYSTL']


In [22]:
print(Dc_reactions)

['3HAD100', '3HAD120', '3HAD121', '3HAD140', '3HAD141', '3HAD160', '3HAD161', '3HAD181', '3HAD40', '3HAD60', '3HAD80', '3OAR121', '3OAR141', '3OAR161', '3OAR181', '3OAS100', '3OAS120', '3OAS121', '3OAS140', '3OAS141', '3OAS160', '3OAS161', '3OAS181', '3OAS60', '3OAS80', 'ACCOAC', 'ACGK', 'ACGS', 'ACHBS', 'ACLS', 'ACODA', 'ADCL', 'ADCS', 'ADMDC', 'ADSK', 'ADSS', 'AGMHE', 'AGPAT160', 'AGPAT161', 'AGPAT181', 'AHCYSNS', 'AIRC2', 'AMMQLT8', 'AMPMS2', 'ANPRT', 'ANS', 'APRAUR', 'ARGSS', 'ASP1DC', 'ASPCT', 'ATPPRT', 'BPNT', 'CBL1abcpp', 'CBL1tonex', 'CDPMEK', 'CHORM', 'CHORS', 'CHRPL', 'CLt3_2pp', 'COBALT2tpp', 'COLIPAabcpp', 'COLIPAtex', 'CS', 'CTPS2', 'CU2tpp', 'CYSS', 'CYSTL', 'DAPDC', 'DASYN160', 'DASYN161', 'DASYN181', 'DB4PS', 'DDPA', 'DHAD1', 'DHAD2', 'DHBS', 'DHDPRy', 'DHDPS', 'DHFS', 'DHNAOT4', 'DHNPA2', 'DHPPDA2', 'DHPS2', 'DHPTDCs', 'DHQS', 'DHQTi', 'DM_4HBA', 'DM_HMFURN', 'DMATT', 'DMQMT', 'DNMPPA', 'DNTPPA', 'DPCOAK', 'DPR', 'DXPRIi', 'DXPS', 'EDTXS1', 'EDTXS2', 'ENTCS', 'EX_ala-D

In [264]:
len(Dc_reactions)

796

In [265]:
IrDc_reaction = Dc_reactions

In [28]:
# find result
rxn_ids_list = []

for reaction in IrDc_reaction + IrEs_reactions:
    if  (reaction in IrDc_reaction and reaction not in IrEs_reactions) or (reaction in IrEs_reactions and reaction not in IrDc_reaction):
        rxn_ids_list.append(reaction)

In [266]:
len(IrEs_reactions)

372

In [30]:
len(rxn_ids_list)

1

In [43]:
# Save Your Results:
with open('Solutions/Task 1/task1_part1.txt', 'w') as f:
    for rxn_id in Ir_reactions:
        f.write("%s\n" % rxn_id.id)

In [32]:
# Save Your Results:
with open('Solutions/Task 1/task1_part1.txt', 'w') as f:
    for rxn_id in IrEs_reactions:
        f.write("%s\n" % rxn_id)

## Part 2:

Use your LP in the previous part, find all partially or fully coupled pairs in our `iAF1260` E.Coli metabolic network in the aerobic medium. You **only** need to investigate the pairs indicated in the `Desirable_Reactions.txt` file. 

**Computational Hint:** These two kinds of coupling have transitivity properties!

In [8]:
task1_model = cobra.io.read_sbml_model("iAF1260.xml")

In [9]:
# including off blocked reactions
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task1_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task1_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0
    
# for reaction in cobra.flux_analysis.find_blocked_reactions(task1_model):
#     task1_model.remove_reactions([reaction])

In [10]:
rxns = []
with open('Desirable_Reactions.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        rxns.append(line.rstrip())

In [11]:
len(rxns)

103

In [12]:
# blocked reactions
blocked_reactions = cobra.flux_analysis.find_blocked_reactions(task1_model)
len(blocked_reactions)

925

In [66]:
essential_reactions = cobra.flux_analysis.find_essential_reactions(task1_model)
len(essential_reactions)

372

In [17]:
# find all coupled reactions in rxns
coupled_pairs_list = []

counter = 0

for reaction2_id in rxns:
    reaction2 = task1_model.reactions.get_by_id(reaction2_id)

    if reaction2_id in blocked_reactions:
        continue
            
    lb = reaction2.lower_bound
    ub = reaction2.upper_bound

    reaction2.lower_bound = -1000
    reaction2.upper_bound = 0
    reaction2.lower_bound = 0

    temp_blocked_reactions = cobra.flux_analysis.find_blocked_reactions(task1_model)

    for reaction1_id in rxns:
        if reaction1_id in temp_blocked_reactions:
            coupled_pairs_list.append((reaction1_id, reaction2_id))

    reaction2.lower_bound = -1000
    reaction2.upper_bound = ub
    reaction2.lower_bound = lb
    
    print(counter)
    counter += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


In [14]:
# find all coupled reactions in rxns
coupled_pairs_list = []

counter = 0

for reaction1_id in rxns:
    reaction1 = task1_model.reactions.get_by_id(reaction1_id)
    print('Reaction ID1: ', reaction1_id)

    for reaction2_id in rxns:
        reaction2 = task1_model.reactions.get_by_id(reaction2_id)

        if reaction1_id in blocked_reactions:
            if reaction2_id not in blocked_reactions:
                coupled_pairs_list.append((reaction1_id, reaction2_id))
                continue
            else:
                continue
        
        if reaction2_id in blocked_reactions:
            continue

        
        lb = reaction2.lower_bound
        ub = reaction2.upper_bound

        reaction2.lower_bound = -1000
        reaction2.upper_bound = 0
        reaction2.lower_bound = 0

        task1_model.objective = reaction1.id
        task1_model.objective_direction = 'min'
        
        if(task1_model.optimize().status == 'infeasible'):
            coupled_pairs_list.append((reaction1_id, reaction2_id))
            reaction2.lower_bound = -1000
            reaction2.upper_bound = ub
            reaction2.lower_bound = lb
            print('###################################################')
            continue

        min_flux = task1_model.optimize().objective_value
        
        task1_model.objective_direction = 'max'
        
        if(task1_model.optimize().status == 'infeasible'):
            coupled_pairs_list.append((reaction1_id, reaction2_id))
            reaction2.lower_bound = -1000
            reaction2.upper_bound = ub
            reaction2.lower_bound = lb
            print('###################################################')
            continue
            
        max_flux = task1_model.optimize().objective_value

        print('Reaction ID2: ', reaction2_id)
        print('Max Flux: ', max_flux)
        print('Min Flux: ', min_flux)

        if min_flux > -1e-3 and max_flux < 1e-3:
            coupled_pairs_list.append((reaction1_id, reaction2_id))

        reaction2.lower_bound = -1000
        reaction2.upper_bound = ub
        reaction2.lower_bound = lb
    
    print(counter)
    counter += 1

Reaction ID1:  MN6PP
Reaction ID2:  MN6PP
Max Flux:  0.0
Min Flux:  0.0
Reaction ID2:  PSSA120
Max Flux:  101.61000000000031
Min Flux:  0.0
Reaction ID2:  SDPDS
Max Flux:  101.61000000000007
Min Flux:  0.0
Reaction ID2:  PSSA161
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  UDCPDP
Max Flux:  101.61000000000034
Min Flux:  0.0
Reaction ID2:  GDPMNP
Max Flux:  101.60999999999999
Min Flux:  0.0
Reaction ID2:  GART
Max Flux:  101.60999999999994
Min Flux:  0.0
Reaction ID2:  KDOCT2
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  ANPRT
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  KDOPP
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  AIRC2
Max Flux:  101.60999999999986
Min Flux:  0.0
Reaction ID2:  CYSabc2pp
Max Flux:  101.61000000000003
Min Flux:  0.0
Reaction ID2:  UPP3MT
Max Flux:  101.60999999999994
Min Flux:  0.0
Reaction ID2:  PRFGS
Max Flux:  101.61000000000001
Min Flux:  0.0
Reaction ID2:  ACCOAC
Max Flux:  101.61000000000027
Min Flux:  0.0
Reaction ID2:  MOHMT
Max Flux:  101.61
M

Reaction ID2:  GDPMNH
Max Flux:  18.106086956521754
Min Flux:  0.0
Reaction ID2:  PTA2
Max Flux:  18.106086956521697
Min Flux:  0.0
Reaction ID2:  AHCYSNS
Max Flux:  18.10608695652172
Min Flux:  0.0
Reaction ID2:  PLIPA2E120pp
Max Flux:  18.10608695652172
Min Flux:  0.0
Reaction ID2:  PGSA120
Max Flux:  18.106086956521782
Min Flux:  0.0
Reaction ID2:  PSD180
Max Flux:  18.106086956521658
Min Flux:  0.0
Reaction ID2:  PSD160
Max Flux:  18.10608695652172
Min Flux:  0.0
Reaction ID2:  G3PAT181
Max Flux:  18.106086956521665
Min Flux:  0.0
Reaction ID2:  PSSA181
Max Flux:  18.10608695652175
Min Flux:  0.0
Reaction ID2:  AGMHE
Max Flux:  18.106086956521786
Min Flux:  0.0
Reaction ID2:  ASPCT
Max Flux:  18.106086956521743
Min Flux:  0.0
Reaction ID2:  PANTS
Max Flux:  18.106086956521743
Min Flux:  0.0
1
Reaction ID1:  SDPDS
Reaction ID2:  MN6PP
Max Flux:  7.677333333333331
Min Flux:  0.0
Reaction ID2:  PSSA120
Max Flux:  7.6773333333333325
Min Flux:  3.2933482588316166e-17
Reaction ID2:  SDPD

Reaction ID2:  G3PAT160
Max Flux:  15.423703703703687
Min Flux:  -2.2784761752683724e-13
Reaction ID2:  IG3PS
Max Flux:  15.423703703703728
Min Flux:  -1.5856628995661544e-16
Reaction ID2:  AMPMS2
Max Flux:  15.423703703703673
Min Flux:  -4.176628526154409e-15
Reaction ID2:  OMBZLM
Max Flux:  15.423703703703726
Min Flux:  5.481492641208743e-13
Reaction ID2:  MAN1PT2
Max Flux:  15.42370370370368
Min Flux:  -5.670516157141535e-16
Reaction ID2:  PSSA160
Max Flux:  15.42370370370377
Min Flux:  -5.773159728050809e-15
Reaction ID2:  3HAD121
Max Flux:  15.423703703703707
Min Flux:  1.688979104936365e-14
Reaction ID2:  DHDPS
Max Flux:  15.423703703703701
Min Flux:  5.4831289963830426e-15
Reaction ID2:  HEX4
Max Flux:  15.423703703703683
Min Flux:  6.313030760161564e-12
Reaction ID2:  GMHEPK
Max Flux:  15.423703703703664
Min Flux:  6.648015574086116e-15
Reaction ID2:  PSSA180
Max Flux:  15.42370370370369
Min Flux:  6.723046333656199e-15
Reaction ID2:  SUCBZS
Max Flux:  15.423703703703735
Min Fl

Reaction ID2:  G3PAT181
Max Flux:  9.443083900226766
Min Flux:  3.2949565887343027e-15
Reaction ID2:  PSSA181
Max Flux:  9.443083900226759
Min Flux:  -2.6928940995930147e-15
Reaction ID2:  AGMHE
Max Flux:  9.443083900226764
Min Flux:  2.197642277099648e-15
Reaction ID2:  ASPCT
Max Flux:  9.44308390022677
Min Flux:  0.0
Reaction ID2:  PANTS
Max Flux:  9.443083900226764
Min Flux:  -7.878649278177118e-15
5
Reaction ID1:  GDPMNP
Reaction ID2:  MN6PP
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  PSSA120
Max Flux:  101.61000000000044
Min Flux:  0.0
Reaction ID2:  SDPDS
Max Flux:  101.61000000000001
Min Flux:  0.0
Reaction ID2:  PSSA161
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  UDCPDP
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  GDPMNP
Max Flux:  0.0
Min Flux:  0.0
Reaction ID2:  GART
Max Flux:  101.61000000000016
Min Flux:  0.0
Reaction ID2:  KDOCT2
Max Flux:  101.61000000000021
Min Flux:  0.0
Reaction ID2:  ANPRT
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  KDOPP
Max Flux:  10

Reaction ID2:  PLIPA2E120pp
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  PGSA120
Max Flux:  101.61000000000024
Min Flux:  0.0
Reaction ID2:  PSD180
Max Flux:  101.60999999999969
Min Flux:  0.0
Reaction ID2:  PSD160
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  G3PAT181
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  PSSA181
Max Flux:  101.61
Min Flux:  0.0
Reaction ID2:  AGMHE
Max Flux:  101.61000000000001
Min Flux:  0.0
Reaction ID2:  ASPCT
Max Flux:  101.61000000000001
Min Flux:  0.0
Reaction ID2:  PANTS
Max Flux:  101.61
Min Flux:  0.0
7
Reaction ID1:  FORtppi
8
Reaction ID1:  KDOCT2
Reaction ID2:  MN6PP
Max Flux:  1.0422709163346615
Min Flux:  0.0
Reaction ID2:  PSSA120
Max Flux:  1.0422709163346624
Min Flux:  -1.9032394707859838e-16
Reaction ID2:  SDPDS
Max Flux:  1.0422709163346653
Min Flux:  2.2119267155180247e-15
Reaction ID2:  PSSA161
Max Flux:  1.0422709163346628
Min Flux:  4.983528412481556e-15
Reaction ID2:  UDCPDP
Max Flux:  1.0422709163346688
Min Flux:  -3.7755134171

...

Reaction ID2:  CYSabc2pp
Max Flux:  22.580000000000005
Min Flux:  0.0
Reaction ID2:  UPP3MT
Max Flux:  22.58
Min Flux:  0.0
Reaction ID2:  PRFGS
Max Flux:  22.579999999999977
Min Flux:  0.0
Reaction ID2:  ACCOAC
Max Flux:  22.57999999999997
Min Flux:  3.446444925357971e-15
Reaction ID2:  MOHMT
Max Flux:  22.579999999999878
Min Flux:  1.050223099014487e-14
Reaction ID2:  HEPK1
Max Flux:  22.579999999999956
Min Flux:  2.012286404116459e-14
Reaction ID2:  PSSA140
Max Flux:  22.580000000000073
Min Flux:  5.100857880571819e-16
Reaction ID2:  HSTPT
Max Flux:  22.580000000000002
Min Flux:  8.522983643347559e-15
Reaction ID2:  GLUPRT
Max Flux:  22.579999999999984
Min Flux:  3.552713678800501e-15
Reaction ID2:  3OAS100
Max Flux:  22.579999999999956
Min Flux:  0.0
Reaction ID2:  NPHS
Max Flux:  22.579999999999984
Min Flux:  0.0
Reaction ID2:  PSD161
Max Flux:  22.57999999999999
Min Flux:  0.0
Reaction ID2:  DASYN120
Max Flux:  1.546040649589774e-31
Min Flux:  0.0
Reaction ID2:  ACGS
Max Flux:  2

Reaction ID2:  PTA2
Max Flux:  18.10608695652178
Min Flux:  0.0
Reaction ID2:  AHCYSNS
Max Flux:  18.10608695652163
Min Flux:  -5.941483864042375e-15
Reaction ID2:  PLIPA2E120pp
Max Flux:  18.106086956521743
Min Flux:  -6.1780361325696225e-15
Reaction ID2:  PGSA120
Max Flux:  18.10608695652174
Min Flux:  -2.583184039559883e-15
Reaction ID2:  PSD180
Max Flux:  0.0
Min Flux:  7.993605777301127e-15
Reaction ID2:  PSD160
Max Flux:  18.10608695652175
Min Flux:  0.0
Reaction ID2:  G3PAT181
Max Flux:  18.106086956521764
Min Flux:  0.0
Reaction ID2:  PSSA181
Max Flux:  18.10608695652174
Min Flux:  0.0
Reaction ID2:  AGMHE
Max Flux:  18.10608695652172
Min Flux:  0.0
Reaction ID2:  ASPCT
Max Flux:  18.10608695652167
Min Flux:  0.0
Reaction ID2:  PANTS
Max Flux:  18.106086956521704
Min Flux:  0.0
89
Reaction ID1:  PSD160
Reaction ID2:  MN6PP
Max Flux:  18.10608695652169
Min Flux:  0.0
Reaction ID2:  PSSA120
Max Flux:  18.106086956521747
Min Flux:  0.0
Reaction ID2:  SDPDS
Max Flux:  18.1060869565

Reaction ID2:  DHDPS
Max Flux:  23.908235294117645
Min Flux:  -1.4362538866197413e-14
Reaction ID2:  HEX4
Max Flux:  23.908235294117645
Min Flux:  6.684769955541746e-13
Reaction ID2:  GMHEPK
Max Flux:  23.908235294117645
Min Flux:  0.0
Reaction ID2:  PSSA180
Max Flux:  23.908235294117656
Min Flux:  -2.614556024633688e-13
Reaction ID2:  SUCBZS
Max Flux:  23.908235294117645
Min Flux:  4.7955655615242195e-32
Reaction ID2:  PRAIS
Max Flux:  23.908235294117645
Min Flux:  0.0
Reaction ID2:  SHSL1
Max Flux:  23.90823529411767
Min Flux:  2.3684757858670005e-15
Reaction ID2:  PSD181
Max Flux:  23.908235294117645
Min Flux:  -8.023595756322945e-14
Reaction ID2:  THZPSN
Max Flux:  23.908235294117674
Min Flux:  -3.1957525893190346e-16
Reaction ID2:  PSD120
Max Flux:  23.90823529411764
Min Flux:  -5.311088123302182e-15
Reaction ID2:  PSD140
Max Flux:  23.908235294117503
Min Flux:  4.426031970500496e-15
Reaction ID2:  PSSA141
Max Flux:  23.908235294117574
Min Flux:  -3.1884377857239274e-11
Reaction I

Reaction ID2:  PSSA161
Max Flux:  1.0578114733383905
Min Flux:  0.0
Reaction ID2:  UDCPDP
Max Flux:  1.0578114733383868
Min Flux:  0.0
Reaction ID2:  GDPMNP
Max Flux:  1.0578114733383897
Min Flux:  0.0
Reaction ID2:  GART
Max Flux:  1.0578114733383883
Min Flux:  0.0
Reaction ID2:  KDOCT2
Max Flux:  0.0
Min Flux:  -1.7763568394002505e-15
Reaction ID2:  ANPRT
Max Flux:  1.0578114733383872
Min Flux:  0.0
Reaction ID2:  KDOPP
Max Flux:  0.0
Min Flux:  1.7763568394002505e-15
Reaction ID2:  AIRC2
Max Flux:  1.0578114733383885
Min Flux:  0.0
Reaction ID2:  CYSabc2pp
Max Flux:  1.057811473338389
Min Flux:  0.0
Reaction ID2:  UPP3MT
Max Flux:  1.0578114733383877
Min Flux:  0.0
Reaction ID2:  PRFGS
Max Flux:  1.0578114733383879
Min Flux:  0.0
Reaction ID2:  ACCOAC
Max Flux:  -8.610262927542102e-16
Min Flux:  -5.967631668605853e-17
Reaction ID2:  MOHMT
Max Flux:  1.0578114733383879
Min Flux:  0.0
Reaction ID2:  HEPK1
Max Flux:  -1.872777041241179e-17
Min Flux:  0.0
Reaction ID2:  PSSA140
Max Flux

Reaction ID2:  PNTK
Max Flux:  16.975451263537913
Min Flux:  -1.2257563912960822e-15
Reaction ID2:  MCTP2App
Max Flux:  16.97545126353793
Min Flux:  -7.745555948375868e-14
Reaction ID2:  PRASCS
Max Flux:  16.975451263537902
Min Flux:  1.425899904295368e-14
Reaction ID2:  CDAPPA120
Max Flux:  16.97545126353788
Min Flux:  1.8182323655639646e-14
Reaction ID2:  CLt3_2pp
Max Flux:  16.97545126353794
Min Flux:  7.043641032752297e-14
Reaction ID2:  GDPMNH
Max Flux:  16.975451263537842
Min Flux:  -9.390174215584843e-15
Reaction ID2:  PTA2
Max Flux:  16.97545126353788
Min Flux:  6.849700624199419e-15
Reaction ID2:  AHCYSNS
Max Flux:  16.975451263537902
Min Flux:  8.770425171784297e-15
Reaction ID2:  PLIPA2E120pp
Max Flux:  16.975451263537924
Min Flux:  9.451849733727148e-15
Reaction ID2:  PGSA120
Max Flux:  16.9754512635379
Min Flux:  3.990334277463744e-15
Reaction ID2:  PSD180
Max Flux:  16.97545126353791
Min Flux:  4.494861273308794e-15
Reaction ID2:  PSD160
Max Flux:  16.97545126353791
Min F

Save your results in a text file with pairs like `('id_1', 'id_2')` in each line separately.

In [69]:
len(coupled_pairs_list)

3625

In [19]:
print(('SDPDS', 'PANTS') in coupled_pairs_list)
print(('PANTS', 'SDPDS') in coupled_pairs_list)

print(('SDPDS', 'MOHMT') in coupled_pairs_list)
print(('MOHMT', 'SDPDS') in coupled_pairs_list)

print(('OMBZLM', 'DHDPS') in coupled_pairs_list)
print(('DHDPS', 'OMBZLM') in coupled_pairs_list)

print(('SDPDS', 'SUCBZS') in coupled_pairs_list)
print(('SUCBZS', 'SDPDS') in coupled_pairs_list)

False
True
False
True
True
False
False
True


In [24]:
coupled_pairs_list_1 = []
for couple in coupled_pairs_list:
    if (couple[1] != couple[0]) and couple[0]<couple[1] and (couple[1], couple[0]) in coupled_pairs_list:
        coupled_pairs_list_1.append((couple[0], couple[1]))

In [66]:
for re in rxns:
    if re not in blocked_reactions:
        coupled_pairs_list_1.append((re, re))

In [25]:
len(coupled_pairs_list_1)

115

In [33]:
for couple1 in coupled_pairs_list_1:
    for couple2 in coupled_pairs_list_1:
        if couple1[1]==couple2[0] and (couple1[0], couple2[1]) not in coupled_pairs_list_1:
            print(couple1)            
            print(couple2)

In [72]:
## Save Your Results:
with open('Solutions/Task 1/task1_part2.txt', 'w') as f:
    for _pair in coupled_pairs_list_1:
        f.write("%s\n" % str(_pair))

# Task 2: QFCA


In this task, you investigate the accuracy of the DCE theorem:  

**DCE Theorem:** In a network without any irreversible blocked reaction, for an arbitrary reaction $R_j$, if $D_j$ is the non-empty set of all the irreversible reactions directionally coupled to $R_j$ (excluding itself), there exists positive coefficients $\{c_d\}_{d: R_d \in D_j}$ such that:

$$
v_j = \sum_{d: R_d \in D_j} c_d v_d
$$ 

held for all feasible flux vectors $v$.  

Considering $R_j$ to be the biomass reaction, find these coefficients for the irreversible reactions directionally coupled to the biomass reaction in the aerobic condition (don't save zero coefficients). Save your result as a dictionary with keys as reaction ids and values as coefficients.

**Hint:** Positive or negative certificates are proper for this job. Also, dual variables could help, accessible in an LP solution by `solution.shadow_prices` related to each metabolite. If necessary, you can have any desirable objective function for the FBA by using `model.objective = mod
el.problem.Objective(expression, direction)`.

**Note:** In the basic model, the (lb, ub) for `ATPM`, `EX_glc(e)`, and `EX_o2(e)` are $(8.39, 8.39)$, $(-10, 1000)$, and $(-20, 1000)$ respectively. For other reactions, (lb, ub) are $(-1000, 1000)$ or $(0, 1000)$.

In [252]:
import numpy as np

In [253]:
task2_model = cobra.io.read_sbml_model("iAF1260.xml")

In [254]:
# including off blocked reactions
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task2_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task2_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0

You can use this script to get a model free of blocked reactions:

In [255]:
task2_consistent_model = cobra.flux_analysis.fastcc(task2_model)

In [257]:
IrEs_reactions = cobra.flux_analysis.find_essential_reactions(task2_consistent_model)

In [258]:
biomass_reaction = task2_consistent_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

In [259]:
Irreversible = []
v_I = []

for reaction in task2_consistent_model.reactions:
    if task2_consistent_model.reactions.get_by_id(reaction.id).lower_bound >= 0:
        Irreversible.append(reaction.id)
        v_I.append(reaction.flux_expression)
        
len_vec = len(Irreversible)
vec_1 = [1]*len_vec
u = [] 
u_0 = []

for i in range(len_vec):
    u.append(task2_consistent_model.problem.Variable('u_'+str(i) , lb=0, ub=1, type='continuous'))
    u_0.append(task2_consistent_model.problem.Constraint(v_I[i] - u[i],lb=0,ub=None))
 

task2_consistent_model.add_cons_vars(u) 
task2_consistent_model.add_cons_vars(u_0)

vbiomass_0 = task2_consistent_model.problem.Constraint(biomass_reaction.flux_expression,lb=0,ub=0)
task2_consistent_model.add_cons_vars(vbiomass_0)

task2_consistent_model.objective = task2_consistent_model.problem.Objective(np.dot(np.transpose(vec_1),u), direction= 'max')

solution = task2_consistent_model.optimize()

print(solution.shadow_prices)
print(solution.fluxes)

12dgr120[c]     -8.700000
12dgr120[p]     -9.700000
12dgr140[c]     -9.500000
12dgr140[p]    -10.500000
12dgr141[c]    126.233333
                  ...    
val-L[c]        -3.300000
val-L[p]        -2.800000
xan[p]         -15.500000
zn2[c]           0.500000
zn2[p]           0.000000
Name: shadow_prices, Length: 1014, dtype: float64
12DGR120tipp    0.0
12DGR140tipp    0.0
12DGR141tipp    0.0
12DGR160tipp    0.0
12DGR161tipp    0.0
               ... 
DCAtpp          0.0
DDCAtpp        -2.5
TTDCAtpp       -2.0
HDCAtpp        -1.5
OCDCAtpp       -2.5
Name: fluxes, Length: 1465, dtype: float64


In [263]:
# Find DCE
Dc_reactions = []
for reaction_id in Irreversible:
    reaction = task2_consistent_model.reactions.get_by_id(reaction_id)
    if reaction.flux > -1e-6 and reaction.flux < 1e-6:
        Dc_reactions.append(reaction_id)
len(Dc_reactions)

796

In [224]:
v_shad = []
for index, value in solution.shadow_prices.items():
    v_shad.append(value)
v_shad = np.array(v_shad)
v_shad.shape

(1014,)

In [244]:
# find Ec_biomass_iAF1260_WT_59p81M reaction place in matrix
i_biomass = 0
for i in range(len(task2_consistent_model.reactions)):
    if task2_consistent_model.reactions[i].id == 'Ec_biomass_iAF1260_WT_59p81M':
        i_biomass = i
        print(i)

1456


In [216]:
S = cobra.util.array.create_stoichiometric_matrix(task2_consistent_model)

In [223]:
S.T.shape

(1464, 1014)

In [227]:
v_shad[..., np.newaxis].shape

(1014, 1)

In [241]:
Lambda = np.matmul(S.T, v_shad)

In [242]:
Lambda.shape

(1464,)

In [245]:
L_biomass = Lambda[i_biomass]
L_biomass

156.6110075999998

In [250]:
coefficients_dict = {}
for i in range(len(task2_consistent_model.reactions)):
    if (Lambda[i] < -1e-6 or Lambda[i] > 1e-6) and i != i_biomass:
        print(Lambda[i])
        coefficients_dict[task2_consistent_model.reactions[i].id] = -Lambda[i]/L_biomass

1.4999999999999982
1.0
1.5000000000000027
1.4999999999999982
1.5
1.5
1.5000000000000018
1.0
1.0
1.0000000000000002
1.0
0.9999999999999996
1.0
1.0
1.0000000000000036
1.0
1.0
1.0000000000000018
1.0000000000000093
1.0
1.0
0.9999999999999858
0.9999999999999858
0.9999999999999996
1.0000000000000036
0.9999999999999751
0.9999999999999929
0.9999999999999929
0.9999999999999876
1.0000000000000426
1.0
0.9999999999999893
0.9999999999999993
0.9999999999999876
1.0000000000000002
1.0000000000000053
1.0
1.0000000000000064
1.0000000000000053
0.9999999999999876
1.0000000000000053
1.0000000000000027
0.999999999999997
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999982
1.0
0.9999999999999982
1.0
1.0
1.0
1.0000000000000036
1.0000000000000018
3.7499999999999876
1.0000000000000124
1.0
0.999999999999976
1.4500000000000712
1.4000000000000035
9.350000000000032
1.4000000000000057
1.0
1.0
1.0000000000000018
0.9999999999999556
0.9999999999999889
2.0000000000000013
1.5000000000000124
0.99999999

In [251]:
## Save your result:
import json
with open('./Solutions/Task 2/task2.json', 'w') as fp:
    json.dump(coefficients_dict, fp)